In [1]:
import pandas as pd
import numpy as np

# 1. Data preprocessing

In [31]:
file_path = 'data.xlsx'

df = pd.read_excel(file_path)

In [5]:
df.head()

,Numéro d'application,Date d'application,Numero de publication,date de publication,CPC,IPC,claim,description
0,17167919,2017-04-25,3264223,2018-01-03,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...","['C08J7-04', 'H04M1-02', 'G06F1-16', 'B23K26-3...","<!-- EPO <DP n=""18""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001..."
1,17166775,2017-04-18,3235458,2017-10-25,"['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...","['A61B17-88', 'A61B17-68', 'A61B17-86']","<!-- EPO <DP n=""20""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">FIE..."
2,17166069,2017-04-11,3249094,2017-11-29,"['D06F58-206', 'D06F58-20']","['D06F58-20', 'D06F58-02']","<!-- EPO <DP n=""22""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001""><b>..."
3,18771363,2018-03-05,3605724,2020-02-05,"['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...","['H01P3-12', 'H01P1-213', 'H01P5-107']","<!-- EPO <DP n=""62""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">Tec..."
4,19881950,2019-11-01,3713245,2020-09-23,"['H04N21-4345', 'H04N21-4821', 'H04N21-4316']","['H04N21-482', 'H04N21-262']","<!-- EPO <DP n=""14""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001..."


check for missing values or duplicates

In [29]:
df.isna().sum()

Numéro d'application     0
Date d'application       0
Numero de publication    0
date de publication      0
CPC                      0
IPC                      0
claim                    0
description              0
dtype: int64

In [11]:
df.duplicated().sum()

0

In [13]:
df.shape

(50000, 8)

In [15]:
df.dtypes

Numéro d'application              int64
Date d'application       datetime64[ns]
Numero de publication             int64
date de publication      datetime64[ns]
CPC                              object
IPC                              object
claim                            object
description                      object
dtype: object

## Nettoyage des colonnes 'CLAIM' et 'DESCRIPTION'

In [33]:
df['claim'].iloc[0]

'<!-- EPO <DP n="18"> --><claim id="c-en-0001" num="0001"><claim-text>A method of manufacturing a cover window for a display panel, comprising:<claim-text>forming a first coating layer (120) on an upper surface of a plastic layer (110);</claim-text><claim-text>performing mechanical processing on an edge of the plastic layer (110) and an edge of the first coating layer (120) to produce a desired shape; and</claim-text><claim-text>performing laser processing on the edge of the first coating layer after performing the mechanical processing.</claim-text></claim-text></claim><claim id="c-en-0002" num="0002"><claim-text>The method of claim 1, wherein an edge of the cover window includes:<claim-text>a side part (310) perpendicular to the upper surface of the plastic layer;</claim-text><claim-text>a first inclined part (320) extending from the side part (310);</claim-text><claim-text>a first parallel part extending from the first inclined part parallel to the upper surface of the plastic layer

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import string
import re

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edema\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\edema\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\edema\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
def remove_html_tags(text):
    clean_text = re.sub('<.*?>', ' ', text)
    return clean_text

def remove_numbers(text):
    clean_text = re.sub(r'[0-9]', '', text)
    return clean_text
    
def remove_row(text):
    clean_text = re.sub('-->','',text)
    return clean_text
    
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
    
def text_lowercase(text):
    return text.lower()

def remove_whitespace(text):
    return  " ".join(text.split())

def clean_text(text):
    text = remove_html_tags(text)
    text = remove_row(text)
    text = remove_punctuation(text)
    text = text_lowercase(text)
    text = remove_numbers(text)
    text = remove_whitespace(text)

    return text


In [15]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\edema\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [53]:
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text


def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None 

def lemm_words(text):
    lemmatizer = WordNetLemmatizer()
    pos_tagged = nltk.pos_tag(text)

    print("POS Tagged:", pos_tagged)

    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    
    lemmatized = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized.append(word)
        else:        
            # else use the tag to lemmatize the token
            lemmatized.append(lemmatizer.lemmatize(word, tag))
            
    return lemmatized

def lemm_text(text):
    text = remove_stopwords(text)
    text = lemm_words(text)
    
    return text
    

In [35]:
df['claim'] = df['claim'].apply(clean_text)

In [37]:
df['claim'].iloc[0]

'a method of manufacturing a cover window for a display panel comprising forming a first coating layer on an upper surface of a plastic layer performing mechanical processing on an edge of the plastic layer and an edge of the first coating layer to produce a desired shape and performing laser processing on the edge of the first coating layer after performing the mechanical processing the method of claim wherein an edge of the cover window includes a side part perpendicular to the upper surface of the plastic layer a first inclined part extending from the side part a first parallel part extending from the first inclined part parallel to the upper surface of the plastic layer and a second inclined part extending from the first horizontal part to the upper surface of the plastic layer and wherein the side part and the first inclined part are formed by mechanical processing the method of claim wherein the first parallel part and the second inclined part are formed by laser processing the m

In [39]:
df['claim'] = df['claim'].apply(lemm_text)

In [41]:
df['claim'].iloc[0]

['method',
 'manufacturing',
 'cover',
 'window',
 'display',
 'panel',
 'comprise',
 'form',
 'first',
 'coat',
 'layer',
 'upper',
 'surface',
 'plastic',
 'layer',
 'perform',
 'mechanical',
 'processing',
 'edge',
 'plastic',
 'layer',
 'edge',
 'first',
 'coat',
 'layer',
 'produce',
 'desire',
 'shape',
 'perform',
 'laser',
 'processing',
 'edge',
 'first',
 'coat',
 'layer',
 'perform',
 'mechanical',
 'processing',
 'method',
 'claim',
 'wherein',
 'edge',
 'cover',
 'window',
 'include',
 'side',
 'part',
 'perpendicular',
 'upper',
 'surface',
 'plastic',
 'layer',
 'first',
 'inclined',
 'part',
 'extend',
 'side',
 'part',
 'first',
 'parallel',
 'part',
 'extend',
 'first',
 'inclined',
 'part',
 'parallel',
 'upper',
 'surface',
 'plastic',
 'layer',
 'second',
 'inclined',
 'part',
 'extend',
 'first',
 'horizontal',
 'part',
 'upper',
 'surface',
 'plastic',
 'layer',
 'wherein',
 'side',
 'part',
 'first',
 'inclined',
 'part',
 'form',
 'mechanical',
 'processing',
 

In [43]:
df['description'] = df['description'].apply(clean_text)

In [45]:
df['description'].iloc[0]

'the present invention relates to a cover window for a display panel and more particularly to a method of manufacturing the same a display device may include a display panel including a plurality of pixels displaying an image and a transparent cover window covering a display side of the display panel the cover window may protect the display panel from external impact and scratches generated in use of the display panel the cover window may include glass or plastic as an example when the cover window includes plastic a hard coating layer may be formed on a surface of the plastic which may protect a surface of the display panel and may increase strength of the cover window the cover window may have any desired shape the shape of the cover window may correspond with a shape of a display panel of a mobile phone as an example the cover window may be manufactured by a mechanical processing method using computer numerical control cnc process machinery when processing the cover window including

In [47]:
df['description'] = df['description'].apply(lemm_text)

In [49]:
df['description'].iloc[0]

['present',
 'invention',
 'relate',
 'cover',
 'window',
 'display',
 'panel',
 'particularly',
 'method',
 'manufacture',
 'display',
 'device',
 'may',
 'include',
 'display',
 'panel',
 'include',
 'plurality',
 'pixel',
 'display',
 'image',
 'transparent',
 'cover',
 'window',
 'cover',
 'display',
 'side',
 'display',
 'panel',
 'cover',
 'window',
 'may',
 'protect',
 'display',
 'panel',
 'external',
 'impact',
 'scratch',
 'generate',
 'use',
 'display',
 'panel',
 'cover',
 'window',
 'may',
 'include',
 'glass',
 'plastic',
 'example',
 'cover',
 'window',
 'include',
 'plastic',
 'hard',
 'coat',
 'layer',
 'may',
 'form',
 'surface',
 'plastic',
 'may',
 'protect',
 'surface',
 'display',
 'panel',
 'may',
 'increase',
 'strength',
 'cover',
 'window',
 'cover',
 'window',
 'may',
 'desire',
 'shape',
 'shape',
 'cover',
 'window',
 'may',
 'correspond',
 'shape',
 'display',
 'panel',
 'mobile',
 'phone',
 'example',
 'cover',
 'window',
 'may',
 'manufacture',
 'mechani

In [57]:
df.head(100)

,Numéro d'application,Date d'application,Numero de publication,date de publication,CPC,IPC,claim,description
0,17167919,2017-04-25,3264223,2018-01-03,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...","['C08J7-04', 'H04M1-02', 'G06F1-16', 'B23K26-3...","[method, manufacturing, cover, window, display...","[present, invention, relate, cover, window, di..."
1,17166775,2017-04-18,3235458,2017-10-25,"['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...","['A61B17-88', 'A61B17-68', 'A61B17-86']","[compression, screw, apply, compression, bone,...","[field, invention, present, disclosure, relate..."
2,17166069,2017-04-11,3249094,2017-11-29,"['D06F58-206', 'D06F58-20']","['D06F58-20', 'D06F58-02']","[clothes, dryer, comprise, case, dry, drum, pr...","[field, present, invention, relate, technical,..."
3,18771363,2018-03-05,3605724,2020-02-05,"['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...","['H01P3-12', 'H01P1-213', 'H01P5-107']","[diplexer, comprise, filter, pair, constitute,...","[technical, field, present, invention, relate,..."
4,19881950,2019-11-01,3713245,2020-09-23,"['H04N21-4345', 'H04N21-4821', 'H04N21-4316']","['H04N21-482', 'H04N21-262']","[method, display, electronic, program, guide, ...","[application, claim, priority, chinese, patent..."
...,...,...,...,...,...,...,...,...
95,18861308,2018-09-28,3690503,2020-08-05,"['E06B9-24', 'G02B5-208', 'G02F1-157', 'G02F1-...","['E06B9-24', 'G02B5-22', 'G02F1-13', 'G02F1-15...","[glass, structure, comprise, pair, glass, plat...","[technical, field, present, invention, relate,..."
96,18754898,2018-02-07,3584446,2019-12-25,"['H05K7-14', 'F04D29-5813', 'H02K11-33', 'F04D...","['F04D29-58', 'H05K7-14', 'F04D29-42', 'F04D13...","[electric, pump, comprise, motor, impeller, ro...","[technical, field, present, invention, relate,..."
97,20778868,2020-02-21,3952365,2022-02-09,"['H04M1-7243', 'H04M2250-66']",['H04W8-18'],"[sim, card, selection, method, apply, terminal...","[crossreference, relate, application, applicat..."
98,20172130,2020-04-29,3904492,2021-11-03,"['C12M47-06', 'C12M45-07']",['C12M1-00'],"[method, pretreating, biomass, particular, bio...","[present, invention, relate, method, pretreati..."


In [59]:
df.to_excel("C:/Users/edema/Documents/Scolarité/MASTERCAMP/Explain/lemm_data.xlsx")